In [1]:
# 569. 员工薪水中位数
import pandas as pd
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
    'company': ['A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C'],
    'salary': [2341, 341, 15, 15314, 451, 513, 15, 13, 1154, 1345, 1221, 234, 2345, 2645, 2645, 2652, 65]
}
df = pd.DataFrame(data)
print(df)

    id company  salary
0    1       A    2341
1    2       A     341
2    3       A      15
3    4       A   15314
4    5       A     451
5    6       A     513
6    7       B      15
7    8       B      13
8    9       B    1154
9   10       B    1345
10  11       B    1221
11  12       B     234
12  13       C    2345
13  14       C    2645
14  15       C    2645
15  16       C    2652
16  17       C      65


In [2]:
def median_employee_salary(df: pd.DataFrame) -> pd.DataFrame:
    company_set=set(df["company"].tolist())   
    id_list=[]
    for c in company_set:  # 相当于partition by操作
        sub_df=df[df["company"]==c].sort_values(by=["salary","id"],ascending=True).reset_index(drop=True)  # 每个区的索引就相当于dense_rank
        if sub_df.shape[0] % 2==0:  # sub_df.shape[0]相当于count
            blower=sub_df.shape[0]/2-1  # 注意index和真实位置区别
            upper=sub_df.shape[0]/2
            medium=sub_df.iloc[[blower,upper],:]["id"].tolist()
        else:
            m=int(sub_df.shape[0]/2)
            medium=pd.DataFrame(sub_df.iloc[m,:]).T["id"].tolist()  # series要变成常见的df需要转置
        id_list.extend(medium)
    
    # condition=df["id"].apply(lambda x:x in id_list)  # 可以用isin()
    condition=df["id"].isin(id_list)
    return df[condition]    

In [3]:
median_employee_salary(df)

id company  salary
4    5       A     451
5    6       A     513
8    9       B    1154
11  12       B     234
13  14       C    2645

In [ ]:
# 方法二：使用中位数特性  正反序
import pandas as pd

def median_employee_salary(employee: pd.DataFrame) -> pd.DataFrame:
    # Calculate 'rnk1' within each company based on 'salary'
    employee['rnk1'] = employee.sort_values(by=['salary', 'id']).groupby('company')['salary'].rank(method='first')

    # Calculate 'rnk2' within each company based on 'salary' in descending order
    employee['rnk2'] = employee.sort_values(by=['salary', 'id'], ascending=False).groupby('company')['salary'].rank(method='first', ascending=False)

    # Count the number of employees within each company
    employee['cnt'] = employee.groupby('company')['salary'].transform('count')

    # Select rows where 'rnk1' is greater than or equal to half the count and 'rnk2' is greater than or equal to half the count
    result = employee[(employee['rnk1'] >= employee['cnt'] / 2) & (employee['rnk2'] >= employee['cnt'] / 2)]

    # Return the DataFrame with 'id', 'company', and 'salary' columns
    return result[['id', 'company', 'salary']]